# Analysis of Living Music Report (Australia)

Imports and compiles data tables from the Living Music Report data folders from github. You will need to specify a path to the data folders for the "Import" section.

Source: [ciaranframe/livingmusicreport](https://github.com/ciaranframe/livingmusicreport)

## Initialization

In [80]:
import pandas as pd
import numpy as np
from pathlib import Path

## Options

To fix encoding errors:

1. Run the script with `generate_error_list = True`
2. Save the error list csv with a different filename and manually enter corrected values (you can set `generate_error_list` to `False`
3. Enter the path to the completed error list in the options below and run the script again

In [121]:
# convert YYYY-MM dates to YYYY-MM-DD with addition of '-01'
convert_dates = True

# generate blank error list (usually used first time only)
generate_error_list = False

# import error list (should be pathlib Path object; leave blank if false)
p_fix_list = Path(("error_list_with_corrections.csv"))

## Import and Consolidation

Orchestra Name (table) > Series (table, multi) > Repertoire

Season can be pulled from SeriesID
Add Program Position when adding in repertoire

In [97]:
# note: enter full path to data folders if they are not in the same folder as this notebook
data_folder = ("LMR/") 

In [115]:
# Combine all the data into one table

encoding_choice = "utf-8"
p = Path(data_folder)
table_list = []

# Names Table
p_orchestra_names = p / "MPAsummary.csv"
df_orchestra_names = pd.read_csv(p_orchestra_names)

for p_orchestra in [x for x in p.iterdir() if x.is_dir()]:
    for p_series in p_orchestra.glob("*.csv"):
        # Series Table
        df_series = pd.read_csv(p_series, encoding="UTF-8", encoding_errors="replace")
        series_orchestra_id = p_series.parts[-2]
        series_season_year = p_series.name[len(series_orchestra_id):len(series_orchestra_id)+4]
        df_series = df_series.assign(MPAinit = series_orchestra_id, SeasonYear = series_season_year)
        
        p_replist_collection = p_orchestra / p_series.name[0:-len("summary.csv")]
        for p_replist in p_replist_collection.glob("*.csv"):
            # Repertoire List Table
            df_replist = (pd.read_csv(p_replist, encoding="UTF-8", encoding_errors="replace")
                          .assign(SeriesID = p_replist.stem))
            df_replist['ProgramPosition'] = list(range(1,1+len(df_replist)))
            # Push table to running list
            table_list.append((df_replist.join(df_series.set_index("SeriesID"), on="SeriesID")
                               .join(df_orchestra_names.set_index("MPAinit"), on="MPAinit")))
            

# stitch the tables together
df_consolidated = pd.concat(table_list, ignore_index=True)

## Fixing Errors

In [101]:
# generate an error list
if generate_error_list:
    errors_present = df_consolidated.applymap(lambda x: u"\uFFFD" in str(x))
    error_list = df_consolidated.to_numpy().flatten()[errors_present.to_numpy().flatten()]
    df_error_list = pd.DataFrame(np.unique(error_list), columns=['Value'])
    df_error_list['Replacement'] = np.nan
    df_error_list.to_csv('error_list.csv',index=False)

In [116]:
# Fixing Encoding Errors - Mostly works, though several errors still don't get matched
# uses the error list imported in options
if p_fix_list.exists():
    df_fix_list = pd.read_csv(p_fix_list, encoding="UTF-8", encoding_errors="replace")
    dict_fix_list = dict(zip(df_fix_list['Value'], df_fix_list['Replacement']))
    df_consolidated = df_consolidated.replace(dict_fix_list)

In [119]:
# Fixing blank rows
df_consolidated = df_consolidated.dropna(subset=['CompName'])

# Format series date
if convert_dates:
    df_consolidated['SeriesDate'] = df_consolidated['SeriesDate'].map(lambda x: str(x) + "-01")

## Export

In [120]:
df_consolidated.to_csv('LMRConsolidated.csv',index=False)